In [41]:
import pandas as pd 
import numpy as np 
import os 
from pathlib import Path 

In [42]:
### Combine the stats and the ratings for each team depending on the name of the team 
stats_summary = pd.read_csv('../data/preprocessing/mens_summary_season_data.csv')
ratings_summary = pd.read_csv('../data/preprocessing/mens_season_ratings.csv')
ratings_summary.drop(columns=['netRating'], inplace=True)
team_spellings = pd.read_csv('../data/MTeamSpellings.csv', encoding='ISO-8859-1')

# Update Spellings for certain teams based on my findings 

# TeamIDs that need to be renamed - Fix joining issues 
# 1107, 1111, 1216, 1271, 1274, 1363, 1366, 1383, 1410, 1419, 1472, 1474
team_spellings.loc[team_spellings['TeamID'] == 1107, 'TeamNameSpelling'] = 'ualbany'
team_spellings.loc[team_spellings['TeamID'] == 1111, 'TeamNameSpelling'] = 'app state'
team_spellings.loc[team_spellings['TeamID'] == 1271, 'TeamNameSpelling'] = 'maryland eastern shore'
team_spellings.loc[team_spellings['TeamID'] == 1274, 'TeamNameSpelling'] = 'miami'
team_spellings.loc[team_spellings['TeamID'] == 1410, 'TeamNameSpelling'] = 'ut rio grande valley'
team_spellings.loc[team_spellings['TeamID'] == 1419, 'TeamNameSpelling'] = 'ul monroe'
team_spellings.loc[team_spellings['TeamID'] == 1472, 'TeamNameSpelling'] = 'st. thomas-minnesota'
team_spellings.loc[team_spellings['TeamID'] == 1474, 'TeamNameSpelling'] = 'queens university'
ratings_summary.loc[ratings_summary['teamID'] == 260, 'team'] = 'san jose st'

# Take data from year 2014 and up because that was what was available in the ratings data
filtered_stats = stats_summary[stats_summary['Season'] >= 2014]

# Merge spellings with team summary stats 
all_spellings = filtered_stats.merge(team_spellings, how='left', left_on=['TeamID'], right_on=['TeamID'])

# Set spellings to all be lower case 
all_spellings['TeamNameSpelling'] = all_spellings['TeamNameSpelling'].str.lower() 
ratings_summary['team'] = ratings_summary['team'].str.lower()

# Combine season stats and season ratings 
combined = all_spellings.merge(ratings_summary,  how='left', left_on=['Season', 'TeamNameSpelling'], right_on=['season', 'team'])

In [45]:
### Look for teams that need to be renamed to be joined together

# Step 1: Check if all 'Team' values are null for each 'TeamID'
team_null_check = combined.groupby('TeamID')['team'].apply(lambda x: x.isnull().all()).reset_index()

# Step 2: Filter for TeamIDs where all records have 'null' in the 'team' column
team_null_check = team_null_check[team_null_check['team'] == True]

# View teams that do not have a matching spelling in the ratings dataset
team_null_check.head(50) 

# TeamIDs that need to be renamed 
# 1107, 1111, 1216, 1271, 1274, 1363, 1366, 1383, 1410, 1419, 1472, 1474

#Team IDs with no associated ranking, set to the lower quartile
# 1216, 1366, 1383

,TeamID,team


In [46]:
### Find the Lower quantile for offensive rating, defensive rating, and srs rating and use that for the three teams that don't have ratings

# Find the rows where the 'team' field is not null for each 'TeamID'
non_null_teams = combined[combined['team'].notnull()]
final_result = non_null_teams.groupby(['Season', 'TeamID']).first().reset_index()

# Find the lower quartile
lower_quartile_offensive = final_result['offensiveRating'].quantile(0.25)
lower_quartile_defensive = final_result['defensiveRating'].quantile(0.25)
lower_quartile_srs = final_result['srs_rating'].quantile(0.25)

# Display the results
print(f'Lower Quartile (25th percentile) for Offensive Rating: {lower_quartile_offensive}')
print(f'Lower Quartile (25th percentile) for Defensive Rating: {lower_quartile_defensive}')
print(f'Lower Quartile (25th percentile) for SRS Rating: {lower_quartile_srs}')

# Use lower quartile values of ratings for 3 teams missing ratings 
# 1216, 1366, 1383
# Assign lower quartile values to the columns for TeamID 1107
combined.loc[combined['TeamID'] == 1216, ['team', 'offensiveRating', 'defensiveRating', 'srs_rating']] = ['hartford', lower_quartile_offensive, lower_quartile_defensive, lower_quartile_srs]
combined.loc[combined['TeamID'] == 1366, ['team', 'offensiveRating', 'defensiveRating', 'srs_rating']] = ['savannah st', lower_quartile_offensive, lower_quartile_defensive, lower_quartile_srs]
combined.loc[combined['TeamID'] == 1383, ['team', 'offensiveRating', 'defensiveRating', 'srs_rating']] = ['st francis ny', lower_quartile_offensive, lower_quartile_defensive, lower_quartile_srs]

### Retrieve final joined dataset 

# Find the rows where the 'team' field is not null for each 'TeamID'
mens_season_data = combined[combined['team'].notnull()]
mens_season_data = mens_season_data.groupby(['Season','TeamID']).first().reset_index()

# Drop unnecessary columns 
mens_season_data.drop(columns=['TeamNameSpelling', 'season', 'teamID', 'team'], inplace=True)

mens_season_data[mens_season_data['TeamID'] == 1186]

Lower Quartile (25th percentile) for Offensive Rating: 102.5
Lower Quartile (25th percentile) for Defensive Rating: 102.9
Lower Quartile (25th percentile) for SRS Rating: -4.6


,Season,TeamID,TeamName,Win_Percentage,Points_Per_Game,FG_Percentage,Threes_Per_Game,Three_Point_Percentage,Free_Throws_Per_Game,Free_Throw_Percentage,Offensive_Rebound_Rate,Defensive_Rebound_Rate,Effective_FG_Percentage,Turnovers_Per_Game,Opp_FG_Percentage,Opp_Three_Point_Percentage,Opp_Free_Throws_Per_Game,Opp_Turnovers_Per_Game,Conf_a_sun,Conf_a_ten,Conf_aac,Conf_acc,Conf_aec,Conf_big_east,Conf_big_sky,Conf_big_south,Conf_big_ten,Conf_big_twelve,Conf_big_west,Conf_caa,Conf_cusa,Conf_gwc,Conf_horizon,Conf_ind,Conf_ivy,Conf_maac,Conf_mac,Conf_meac,Conf_mid_cont,Conf_mvc,Conf_mwc,Conf_nec,Conf_ovc,Conf_pac_ten,Conf_pac_twelve,Conf_patriot,Conf_sec,Conf_southern,Conf_southland,Conf_summit,Conf_sun_belt,Conf_swac,Conf_wac,Conf_wcc,offensiveRating,defensiveRating,srs_rating
79,2014,1186.0,E Washington,0.448276,75.620690,44.858689,21.758621,37.242472,23.172414,69.345238,29.941292,69.830827,51.924233,12.275862,44.964871,35.028249,24.655172,10.344828,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,111.4,111.9,-2.5
430,2015,1186.0,E Washington,0.741935,79.354839,46.817930,25.161290,39.615385,20.741935,71.384137,29.378531,69.426152,55.368013,11.000000,45.286195,38.362761,20.225806,12.258065,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,110.1,106.6,4.8
781,2016,1186.0,E Washington,0.500000,79.666667,48.109366,26.533333,38.065327,21.066667,68.512658,26.785714,70.964467,56.922629,11.533333,49.216483,37.636761,22.133333,11.366667,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,111.3,113.6,1.0
1132,2017,1186.0,E Washington,0.645161,78.741935,47.397564,21.322581,36.611195,20.516129,76.572327,27.686350,72.231986,54.097453,12.225806,43.941842,37.053571,22.387097,10.290323,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,111.3,113.6,1.0
1483,2018,1186.0,E Washington,0.562500,74.312500,46.619021,23.312500,38.605898,16.062500,76.653696,21.853659,75.000000,54.535459,11.718750,44.657534,35.620915,19.968750,11.531250,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,130.6,128.4,2.7
1834,2019,1186.0,E Washington,0.454545,71.757576,42.791878,25.878788,34.777518,16.090909,72.504708,24.620573,76.408451,50.329949,12.030303,45.121951,37.449664,19.909091,11.727273,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,103.1,105.6,-1.7
2187,2020,1186.0,E Washington,0.724138,77.793103,44.683196,26.551724,34.415584,18.517241,68.715084,27.297794,72.965388,51.983471,13.103448,43.831723,34.317343,20.413793,13.724138,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,109.0,104.2,4.0
2535,2021,1186.0,E Washington,0.681818,77.954545,46.949807,23.318182,35.867446,17.909091,79.949239,22.085890,75.761589,54.054054,11.727273,41.928251,31.645570,16.818182,11.681818,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,110.3,102.6,7.5
2887,2022,1186.0,E Washington,0.516129,75.258065,45.128779,24.483871,35.968379,19.096774,75.675676,23.676012,76.315789,52.771557,12.548387,43.329776,33.806452,18.709677,10.677419,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,103.8,108.4,0.1
3245,2023,1186.0,E Washington,0.677419,75.354839,48.678802,24.129032,35.695187,17.612903,75.274725,26.820809,76.162216,56.517910,13.225806,42.794521,32.537688,17.419355,10.258065,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,107.5,106.4,3.3


In [47]:
## Add team metrics to regular season games to create dataset for supervised ML model
pd.set_option("display.max_columns",None)

mens_reg_season_data = pd.read_csv('../data/MRegularSeasonCompactResults.csv')
mens_reg_season_data = mens_reg_season_data[mens_reg_season_data['Season'] >= 2014]

# Duplicate data to there is a record for each losing and winning team 
mens_reg_season_data['Team1'] = mens_reg_season_data['WTeamID']
mens_reg_season_data['Team2'] = mens_reg_season_data['LTeamID']
mens_reg_season_data['Team1_Wins'] = 1 

flipped = mens_reg_season_data.copy() 
flipped['Team1'], flipped['Team2'] = flipped['Team2'], flipped['Team1'] 
flipped['Team1_Wins'] = 0 

games = pd.concat([mens_reg_season_data, flipped])
games.drop(columns=['WTeamID','WScore','LTeamID','LScore','WLoc','NumOT'], inplace=True)

# Merge games dataset with teams stats 
full_games = games.merge(mens_season_data, left_on=['Season','Team1'], right_on=['Season', 'TeamID'], how='left')
full_games = full_games.rename(columns={col: col + "_1" for col in mens_season_data.columns if col not in ["Season", "TeamID"]})

full_games = full_games.merge(mens_season_data, left_on=['Season','Team2'], right_on=['Season', 'TeamID'], how='left')
full_games = full_games.rename(columns={col: col + "_2" for col in mens_season_data.columns if col not in ["Season", "TeamID"]})
full_games.drop(columns=['TeamID_x', 'TeamName_1', 'TeamID_y', 'TeamName_2'], inplace=True)

In [48]:
output_dir = Path("..") / "data" / "modeling"
output_dir.mkdir(parents=True, exist_ok =True)
output_path = output_dir / "reg_season_ml.csv"
full_games.to_csv(output_path, index=False)